In [2]:
import numpy as np

# OFDM Parameters
N_FFT = 64 # FFT size
N_GUARD = 16 # Guard interval size 

# Generate random data and modulate 
data_symbols = np.random.randint(0, 4, N_FFT) # QPSK modulation
data_mod = (2*data_symbols-1) + 1j*(2*data_symbols-1) 

# Create OFDM frame with IFFT
ofdm_time = np.fft.ifft(data_mod)  

# Add cyclic prefix 
ofdm_cp = np.hstack([ofdm_time[-N_GUARD:], ofdm_time])

# Frequency offset/Doppler shift
offset_hz = 10 # Hz 
offset = offset_hz * 2*np.pi/N_FFT   

# Apply frequency shift
offset_arr = offset*np.arange(len(ofdm_cp))
ofdm_received = ofdm_cp * np.exp(1j*offset_arr)

# Synchronization - Find cyclic prefix to align FFT window
corr = np.convolve(ofdm_received, np.flip(ofdm_received[-N_GUARD:]))
start = np.argmax(np.abs(corr))

# Get FFT window and frequency correction
sync_offset = offset*np.arange(start+N_FFT-start)
sync_sig = ofdm_received[start:start+N_FFT] * np.exp(-1j*sync_offset)

# Decode
data_rx = np.fft.fft(sync_sig)  
data_demod = (np.angle(data_rx)/np.pi) >= 0

print(np.all(data_symbols==data_demod)) # Check if matches

False
